In [1]:
!git clone https://github.com/satproject/neuralheuristicsforsat.git
%cd neuralheuristicsforsat

Cloning into 'neuralheuristicsforsat'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 45 (delta 6), reused 6 (delta 1), pack-reused 25
Unpacking objects: 100% (45/45), done.
/content/neuralheuristicsforsat


In [0]:
!pip install -r requirements.in 

     |████████████████████████████████| 7.8MB 2.5MB/s 
     |████████████████████████████████| 83.1MB 57kB/s 
     |████████████████████████████████| 266kB 43.6MB/s 
     |████████████████████████████████| 3.1MB 45.5MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
  Created wheel for python-sat: filename=python_sat-0.1.5.dev10-cp36-cp36m-linux_x86_64.whl size=1308334 sha256=9e53d824e5e4af48bae9d7c2490cae5c026bfb0a6e0525e855c70157767585a7
  Stored in directory: /root/.cache/pip/wheels/31/99/74/0eedba49c633feb3be67b1a93f34c35c297e329263c8b5ad2f
Successfully built python-sat


In [0]:
%tensorflow_version 1.12

In [0]:
!pip install torch
!pip install numpy

In [14]:
!python dump_dataset_2sat.py -o 100 -c 1000 -j 21021

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [0]:
#@title
import numpy as np
import random
import torch
import matplotlib.pyplot as plt
from tqdm import trange

# use GPU to speed up SimCIM 
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


def data_to_matrix(m, n, data):
    matrix = np.zeros((m, n), dtype=np.int32)
    for j in range(m):
        var1, var2 = data[j][0], data[j][1]
        matrix[j][abs(var1)-1] = var1//abs(var1)
        matrix[j][abs(var2)-1] = var2//abs(var2)
    return matrix

def ising_params(m, n, matrix):
    
    """ Encoding SAT matrix to Ising 2-body Hamiltonian folowing 
        S.Santra et al. Max 2-SAT with up to 108 qubits (2014).""" 
    
    v = np.zeros((m, n),dtype=np.int32)
    for i in range(m):
        expr = matrix[i]
        for index, k in enumerate(expr):
            if k!=0: v[i,index] = k//abs(k) 
                
    # constructing Ising matrix and biases
    J = np.zeros((n, n), dtype=np.float32)
    h = np.zeros(n,dtype=np.float32)
    
    for j in range(m):
        index1 = -1
        index2 = 0
        for i in range(n):
            if v[j,i]!=0 and index1==-1:
                index1 = i+1
                continue
            if v[j,i]!=0 and index1!=0: 
                index2 = i+1
                break
        J[index1-1,index2-1] += v[j,index1-1]*v[j,index2-1]
        h[index1-1] += -v[j,index1-1] 
        h[index2-1] += -v[j,index2-1]  
    return J, h


def ampl_inc(J, b, c, zeta, p, sigma, attempt_num, dt):
    """
    Increments and initializes spins
    """
    return (p*c + zeta*(2*torch.mm(J, c) + b))*dt +\
                     (sigma*torch.zeros((c.size(0), attempt_num), dtype=torch.float32, device=device))*dt

def init_ampl(dim, attempt_num):
    return torch.randn((dim, attempt_num), dtype=torch.float32)


def energy_calculation(J, b, step1, dt, sigma, alpha, zeta, offset, dim, attempt_num, N, c_th, m):
    """ 
    Calculates Ising energy according to 
    S.Santra et al. Max 2-SAT with up to 108 qubits (2014). and
    SimCIM annealer from 
    E. S. Tiunov et al. Annealing by simulating the coherent Ising machine (2019).
    Here is used linear pump function.
    """
    N = int(N)
    attempt_num = int(attempt_num)
    c_current = init_ampl(dim, attempt_num).to(device) 
    dc_momentum = torch.zeros((dim, attempt_num),dtype=torch.float32,device=device)
    init_lambda = np.array([offset + step1*i/float(N) for i in range(N)])
    
    for i in range(1,N):
        dc = ampl_inc(J, b, c_current, zeta, init_lambda[i], sigma, attempt_num, dt)
        dc_momentum = alpha*dc_momentum + (1-alpha)*dc
        dc_momentum/=(1.-alpha**i)
        c1 = c_current + dc_momentum
        th_test = (torch.abs(c1)<c_th).type(torch.float32)
        c_current = c_current + th_test*dc_momentum
        spins_current = torch.sign(c_current)
        
    return (torch.einsum('ij,ik,jk->k',(J,spins_current,spins_current)) +
            torch.einsum('ij,ik->k',(b,spins_current)))*0.25 - 0.25*m
# initial params + hyperparameters


In [0]:
def SIMCim(params, n, m, input_2cnf ):
  #n = 50 # number of variables in 2-SAT formula

  # SimCIM hyperparameters
  c_th = 1.

  #m = len(train_set['cnf']) // 2
  #n = 1000
  
  matrix = data_to_matrix(m, n, input_2cnf)
  J, h = ising_params(m, n, matrix)
  lambda_max = abs(np.max(np.linalg.eigvals(-J)))
  J = torch.tensor(-J, dtype=torch.float32, device=device)
  b = torch.tensor(-h, dtype=torch.float32, device=device).unsqueeze(1)
  value = torch.max(energy_calculation(J, b, lambda_max, params['dt'], params['sigma'], params['alpha'], params['zeta'], 
                            -lambda_max, J.size(0), params['attempt_num'], params['N'], c_th, m))
  satis_simcim = int(value.cpu().numpy()==0) # checking satisfactory
  return satis_simcim


In [0]:
import itertools
import os
import tensorflow as tf
import csv
import time

def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(list(map(int, seq[int(last):int(last + avg)])))
        last += avg

    return out


def predict(params):
    n = 100
    tfrecord_location = '/content/neuralheuristicsforsat/sr_{0}'.format(n)
    name = "train_21021_sr_{0}.tfrecord".format(n)
    filename = os.path.join(tfrecord_location, name)

    record_iterator = tf.python_io.tf_record_iterator(path=filename)
    preds = []
    #targes = []
    #batch_size = n

    train_set = {'cnf': list(), 'sat': list()}
    sim_results = list()

    for string_record in itertools.islice(record_iterator, 100):
        example = tf.train.Example()
        example.ParseFromString(string_record)
        

        m = len(example.features.feature["inputs"].float_list.value) // 2
        
        inputs = chunkIt(example.features.feature["inputs"].float_list.value, m) #split examp: [0,1,2,3,4,5] into [0,1], [2,3], [4,5]
        
        train_set['cnf'].append(inputs)
        targ = int(example.features.feature["sat"].float_list.value[0])
        train_set['sat'].append(targ)

        start = time.time()
        sim_result = SIMCim(params, n, m, inputs) #predict SIMCim, inputs- params ISING Model, n- variables, M-clausures

        end = time.time()
        
        
        logs = open("logs_{0}_{1}.csv".format(n, time.strftime("%Y%m%d")), 'a')
        with logs:
            writer = csv.writer(logs)
            writer.writerows([[m, m/n, sim_result, targ, end - start]])
        sim_results.append(sim_result)
        

    return (sim_results, train_set)



In [0]:
from sklearn.metrics import mean_squared_error 

import csv
from hyperopt import STATUS_OK
from timeit import default_timer as timer
import numpy as np 

def objective(params):
  # Keep track of evals
  global ITERATION


  ITERATION += 1

  start = timer()

  Y_pred, train_set = predict(params)

  run_time = timer() - start

  #todo implemented seed
  loss = np.square(np.subtract(train_set['sat'], Y_pred)).mean() 
  
  out_file = 'gbm_trials.csv'
  of_connection = open(out_file, 'a')
  writer = csv.writer(of_connection)
  writer.writerow([loss, params, ITERATION, run_time])

  return {'loss': loss, 'params': params, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}



In [0]:
# Hyperparameter grid  [pump parametrization (linear, tanh, etc.), number of iterations (N), noise level (sigma), learning rate (dt), coupling (zeta), sample size (attempt_num)] 
 # c_th = 1.  #
 # N = 400 #
 # zeta = 1. #
 # attempt_num = 1000
 # dt = 0.3
 # sigma = 0.2
 # alpha = 0.9 
from hyperopt import hp
from hyperopt.pyll.stochastic import sample

param_grid = {
    'class_weight': [None, 'balanced'],
    'pump_parametrization': ['linear','tanh'],
    #'boosting_type': ['gbdt', 'goss', 'dart'],
    'N': list(range(300, 500, 10)), #num_of_iteraction
    'sigma': list(np.linspace(0, 1)), #noise level
    'dt': list(np.logspace(np.log(0.005), np.log(0.5), base = np.exp(1), num = 1000)), #learning rate
    'attempt_num': list(range(950, 1100, 5)), #sample_size
    'zeta': list(np.linspace(0, 2)) #coupling

}

space = {
    #'class_weight': hp.choice('class_weight', [None, 'balanced']), @TODO
    'N': hp.quniform('N', 300, 500, 10),
    'sigma': hp.uniform('sigma', 0.0, 1.0),
    'dt': hp.loguniform('dt', np.log(0.005), np.log(0.5)),
    'zeta': hp.uniform('zeta', 0.0, 2.0),
    'attempt_num': hp.quniform('attempt_num', 950, 1100, 5),
    'alpha': hp.uniform('alpha', 0.1, 0.9)
}

# Subsampling (only applicable with 'goss')
subsample_dist = list(np.linspace(0.5, 1, 100))

In [0]:
#params = {key: random.sample(value, 1)[0] for key, value in param_grid.items()}

In [79]:
from hyperopt import fmin
from hyperopt import Trials
from hyperopt import rand, tpe


tpe_trials = Trials()
rand_trials = Trials()

global  ITERATION

ITERATION = 0

# Create the algorithms
tpe_algo = tpe.suggest
rand_algo = rand.suggest

rand_best = fmin(fn=objective, space=space, algo=rand_algo, trials=rand_trials, 
                 max_evals=5)

0
{'N': 300.0, 'alpha': 0.8740721382353519, 'attempt_num': 985.0, 'dt': 0.2618371203033544, 'sigma': 0.5841316520969193, 'zeta': 1.9954779920902594}
0
{'N': 300.0, 'alpha': 0.8740721382353519, 'attempt_num': 985.0, 'dt': 0.2618371203033544, 'sigma': 0.5841316520969193, 'zeta': 1.9954779920902594}
1
{'N': 300.0, 'alpha': 0.8740721382353519, 'attempt_num': 985.0, 'dt': 0.2618371203033544, 'sigma': 0.5841316520969193, 'zeta': 1.9954779920902594}
1
{'N': 300.0, 'alpha': 0.8740721382353519, 'attempt_num': 985.0, 'dt': 0.2618371203033544, 'sigma': 0.5841316520969193, 'zeta': 1.9954779920902594}
1
{'N': 300.0, 'alpha': 0.8740721382353519, 'attempt_num': 985.0, 'dt': 0.2618371203033544, 'sigma': 0.5841316520969193, 'zeta': 1.9954779920902594}
0
{'N': 300.0, 'alpha': 0.8740721382353519, 'attempt_num': 985.0, 'dt': 0.2618371203033544, 'sigma': 0.5841316520969193, 'zeta': 1.9954779920902594}
0
{'N': 300.0, 'alpha': 0.8740721382353519, 'attempt_num': 985.0, 'dt': 0.2618371203033544, 'sigma': 0.584